In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pickle
import multiprocessing as mp
import pandas as pd
from sklearn import datasets
import quante
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern
import time
import warnings
import numpy as np



In [ ]:
iris = datasets.load_iris()
pd.DataFrame(iris.data).to_csv('X.csv', index=False)
with open('y.csv', 'w') as f:
    f.write(','.join([str(x) for x in iris.target]))

In [ ]:
if __name__ == '__main__':
    
    def instance(arch):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = MLPClassifier(solver='sgd', max_iter=500,
                        hidden_layer_sizes=arch[:3], alpha=arch[3], random_state=1)
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        return cvs
        
#    print("Call a single instance of MLP Classifier") 
#    print(instance((3,3)))
    
    hp_tune = quante.carlo(instance, [[2, 16], [2, 4], [2, 16], [.01, .99]], kernel=DotProduct()+ WhiteKernel(), n_batches=100, n_processors=4, n_iterations=5)
    
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()




In [ ]:
summary = session.summary()
summary['iteration_id'] = iterations
summary

In [ ]:
n = summary[['score', 'iteration_id']].groupby('iteration_id').mean().plot()


In [ ]:
rounded_alpha=  [x[:3]+(round(x[3],3),) for x in summary['layers']]

In [ ]:

fig = plt.figure(figsize =(13, 6))
ax = fig.add_subplot(111)
ax.boxplot(session.score_history)
ax.set_xticklabels(rounded_alpha)
plt.xticks(rotation=60)
bx = plt.show


In [ ]:
if __name__ == '__main__':
    
    def instance(hp_parameter):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = LogisticRegression(solver='saga', penalty = 'elasticnet', l1_ratio=hp_parameter[0], max_iter=200 )
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        
        return cvs
        
    
    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()


In [ ]:
if __name__ == '__main__':
    
    def instance(hp_parameter):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = LogisticRegression(solver='saga', penalty = 'elasticnet', l1_ratio=hp_parameter[0], max_iter=200 )
        with warnings.catch_warnings(action='ignore'):
            cvs = cross_val_score(clf, X, y)
        
        return cvs
        
    
    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session, iterations = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()


In [ ]:
summary = session.summary()
summary['iteration_id'] = iterations
summary

In [ ]:
n = summary[['score', 'iteration_id']].groupby('iteration_id').mean().plot()
